In [4]:
import numpy as np
import cv2 as cv
import glob

checkboard = (12, 8)
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((checkboard[0]*checkboard[1],3), np.float32)
objp[:,:2] = np.mgrid[0:checkboard[0],0:checkboard[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, checkboard, None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
    
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
    
    # Draw and display the corners
    cv.drawChessboardCorners(img, checkboard, corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(500)
    
cv.destroyAllWindows()

In [7]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [9]:
print("Camera matrix: ") 
print(mtx) 
  
print("\nDistortion coefficient: ") 
print(dist) 
  
print("\nRotation Vectors: ") 
print(rvecs) 
  
print("\nTranslation Vectors: ") 
print(tvecs) 

Camera matrix: 
[[2.89177797e+03 0.00000000e+00 2.00778164e+03]
 [0.00000000e+00 3.20722869e+03 2.17642791e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion coefficient: 
[[ 0.02397363  0.15402835  0.0113913  -0.00693751 -0.36087047]]

Rotation Vectors: 
(array([[ 0.11578214],
       [-0.70944961],
       [-0.86183805]]), array([[ 0.40115211],
       [-0.54986127],
       [-1.2138624 ]]), array([[ 0.22019671],
       [-0.33099685],
       [-1.31472251]]), array([[-0.58083528],
       [-0.10216861],
       [-0.26129761]]), array([[-0.59288684],
       [-0.24952459],
       [-0.44282175]]), array([[-0.50175789],
       [-0.13882747],
       [-0.6657532 ]]), array([[-0.47936219],
       [ 0.13090956],
       [ 0.34910175]]), array([[-0.43548016],
       [ 0.23363831],
       [ 0.63265136]]), array([[-0.61978019],
       [-0.18838701],
       [-0.28233952]]), array([[-0.57444914],
       [-0.24427189],
       [-0.41957989]]))

Translation Vectors: 
(array([[-5.78630437],
  

In [18]:
img = cv.imread('IMG-9953.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [19]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True